<a href="https://colab.research.google.com/github/Chuck2Win/text-generation---torchtext/blob/master/Text_Classification_with_TorchText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install --upgrade torchtext #0.5.0으로 업그레이드

     |████████████████████████████████| 81kB 10.3MB/s 
     |████████████████████████████████| 1.0MB 29.5MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [8]:
# Datasets supports the ngrams method

import torch
import torchtext
import torch.nn as nn
import torch.nn.functional as F
from torchtext.datasets import text_classification
import os

print(torchtext.__version__) # 0.31 -> upgrade 필요 -> upgrade
print(torch.__version__) # 1.4.0 
NGRAMS=2
if not os.path.isdir('./.data'):
    os.mkdir('./.data')
train_dataset,test_dataset=text_classification.DATASETS['AG_NEWS'](root='./.data',ngrams=NGRAMS,vocab=None)
# 이렇게 ngrams=NGRAMS로 설정하면, dataset은 a list of single words + bi-grams string
BATCH_SIZE=16
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

0.5.0
1.4.0


120000lines [00:07, 17042.01lines/s]
120000lines [00:15, 7682.18lines/s]
7600lines [00:00, 7944.43lines/s]


In [0]:
# 공부용
tt=nn.EmbeddingBag(10,3)
tt.forward(torch.LongTensor([0,1,2,3]),torch.LongTensor([0]))
list(tt.parameters())

In [0]:
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

EmbeddingBag : Computes sums or means of bags of embeddings without instantiating the intermediate embeddings
(num of embeddings, embedding dim)

inputs : input(Long Tensor), offsets(Long Tensor,optional)

1) if input is 2D of shape (B,N)

it will be treated as B bags(sequences) each of fixed length N, and this will return B

values aggregated in a way depending on the mode. offsets is ignored and required to be

None in this case


**2)** If input is 1D of shape (N)
It will be treated as a concatenation of multiple bags(sequences). offsets is required

to be a 1D tensor containing the starting index positions of each bag in input. Therefore,

for offsets of shape (B), input will be viewed as having (B) bags. Empty bags(having 0 length)

will have returned vectors filled by zeros.



In [0]:
# AG_NEWS dataset
# 1 : World
# 2 : Sports
# 3 : Business
# 4 : Sci/Tec

In [0]:
vocab_size=len(train_dataset.get_vocab()) # including single word and ngrams
embed_dim=32
num_class=len(train_dataset.get_labels())
model=TextSentiment(vocab_size,embed_dim,num_class).to(device)

In [0]:
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]
    # torch.Tensor.cumsum returns the cumulative sum
    # of elements in the dimension dim.
    # torch.Tensor([1.0, 2.0, 3.0]).cumsum(dim=0) -> 1, 3, 6

    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    # 가령 0,10,2,3 -> 즉 첫번째 문장의 길이가 10, 두번째가 2인 경우
    # 0,10,12 이 batch의 시작이란 것을 알려준다는 것이다.

    text = torch.cat(text)
    return text, offsets, label

10720

In [0]:
from torch.utils.data import DataLoader
# torch.utils.datra.sampler.SubsetRandomSampler(index~)
# torch.utils.data.DataLoader(~, sampler=train_indice)

def train_func(sub_train_):

    # Train the model
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)
    for i, (text, offsets, cls) in enumerate(data): # cls는 class 즉 label을 의미한다.
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # Adjust the learning rate
    scheduler.step()

    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)


In [0]:
import time
from torch.utils.data.dataset import random_split
start_time = time.time()